# Machine learning with TensorFlow: CNN

Date: August 21-23, 2018

Neural network: convolution, pooling, fully-connected and softmax

Classification of human body motion:
- walking
- sitting down
- turning right while walking
- turning left while walking

In [144]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

## Preparing data set for training CNN

In [145]:
SAMPLE_NUMS = 260  # per sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

TRAINING_SET = [['./20180821_walking_straight.csv', 0],
               ['./20180821_sitting.csv', 1],
               ['./20180821_turning_left.csv', 2],
               ['./20180821_turning_right.csv', 3],                
               ['./20180823_walking_straight.csv', 0],
               ['./20180823_sitting.csv', 1],
               ['./20180823_turning_left.csv', 2],
               ['./20180823_turning_right.csv', 3]]

TEST_SET = [['./20180822_walking_straight.csv', 0],
            ['./20180822_sitting.csv', 1],
            ['./20180822_turning_left.csv', 2],
            ['./20180822_turning_right.csv', 3]]

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

In [163]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df, lpf=False):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    if lpf:
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
        CUTOFF = 10.0
        b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
        df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [164]:
training_set = []
test_set = []

def label(l):
    ll = [0,0,0,0]
    ll[l] = 1
    return ll

for file, l in TRAINING_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        training_set.append([df[df['id']==i], label(l)])
        
for file, l in TEST_SET:
    df = pd.read_csv(file, dtype=np.int16)
    conv(df, lpf=True)
    for i in range(MEASUREMENTS):
        test_set.append([df[df['id']==i], label(l)])
    
random.shuffle(training_set)
random.shuffle(test_set)

In [165]:
# use accel x axis and gyro z axis values for classification of human body motion
train_x, train_t = [], []
test_x, test_t = [], []
for df, label in training_set:
    values = []
    for index, item in df.iterrows():
        values.append([item['ax'],item['ay'],item['az'],item['gx'],item['gy'],item['gz']])
    train_x.append(values)
    train_t.append(label)
for df, label in test_set:
    values = []
    for index, item in df.iterrows():
        values.append([item['ax'],item['ay'],item['az'],item['gx'],item['gy'],item['gz']])                     
    test_x.append(values)
    test_t.append(label)

## TensorFlow

In [166]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180823)
tf.set_random_seed(20180823)

In [167]:
num_samples = SAMPLE_NUMS
num_classes = 4

num_layers = 6
num_filters = 12
filter_width = 16

In [168]:
x = tf.placeholder(tf.float32, [None, num_samples, num_layers])
W_conv = tf.Variable(tf.truncated_normal([filter_width,num_layers,num_filters], stddev=0.1))
h_conv = tf.nn.conv1d(x, W_conv, stride=1, padding='SAME')

In [169]:
h_flat = tf.reshape(h_conv, [-1, num_samples*num_filters])

num_units1 = num_samples*num_filters
num_units2 = 64

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.sigmoid(tf.matmul(h_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, num_classes]))
b0 = tf.Variable(tf.zeros([num_classes]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

In [170]:
t = tf.placeholder(tf.float32, [None, num_classes])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [171]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [172]:
i = 0
for _ in range(1500):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:test_x, t:test_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 24.787986755371094, Accuracy: 0.84375
Step: 200, Loss: 16.213350296020508, Accuracy: 0.875
Step: 300, Loss: 13.3768310546875, Accuracy: 0.875
Step: 400, Loss: 11.219956398010254, Accuracy: 0.875
Step: 500, Loss: 10.138622283935547, Accuracy: 0.875
Step: 600, Loss: 9.40229606628418, Accuracy: 0.875
Step: 700, Loss: 8.866084098815918, Accuracy: 0.875
Step: 800, Loss: 8.457115173339844, Accuracy: 0.90625
Step: 900, Loss: 8.134315490722656, Accuracy: 0.90625
Step: 1000, Loss: 7.872710227966309, Accuracy: 0.90625
Step: 1100, Loss: 7.656188011169434, Accuracy: 0.90625
Step: 1200, Loss: 7.473887920379639, Accuracy: 0.90625
Step: 1300, Loss: 7.318158149719238, Accuracy: 0.90625
Step: 1400, Loss: 7.1506452560424805, Accuracy: 0.90625
Step: 1500, Loss: 6.634124279022217, Accuracy: 0.90625


In [173]:
p_test = sess.run(p, feed_dict={x:test_x})
(p_test*100).astype(int)[:10]

array([[60, 24,  3, 11],
       [ 0,  0, 98,  0],
       [ 0, 99,  0,  0],
       [ 3,  0, 96,  0],
       [ 0, 98,  0,  1],
       [ 2,  0,  0, 97],
       [ 3, 91,  4,  0],
       [40, 13, 43,  2],
       [ 1,  0,  0, 98],
       [ 2,  0,  0, 97]])

In [174]:
test_t[:10]

[[1, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1]]

In [175]:
sess.close()